In [1]:
import numpy as np # linear algebra
import pandas as pd # pandas for dataframe based data processing and CSV file I/O
import requests # for http requests
from bs4 import BeautifulSoup # for html parsing and scraping
import bs4
from fastnumbers import isfloat 
from fastnumbers import fast_float
from multiprocessing.dummy import Pool as ThreadPool 

import matplotlib.pyplot as plt
import seaborn as sns
import json
from tidylib import tidy_document # for tidying incorrect html
import datetime as dt

sns.set_style('whitegrid')
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
def get_children(html_content):
    return [item for item in html_content.children if type(item)==bs4.element.Tag or len(str(item).replace("\n","").strip())>0]

def get_table_simple(table,is_table_tag=True):
    elems = table.find_all('tr') if is_table_tag else get_children(table)
    table_data = list()
    for row in elems:
        row_data = list()
        row_elems = get_children(row)
        for elem in row_elems:
            text = elem.text.strip().replace("\n","")
            text = remove_multiple_spaces(text)
            if len(text)==0:
                continue
            row_data.append(text)
        table_data.append(row_data)
    return table_data

def remove_multiple_spaces(string):
    if type(string)==str:
        return ' '.join(string.split())
    return string

In [3]:
url = "https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=billRatesYear&year={0}"
years = list(range(2002,2020))

In [4]:
all_query_df = None
for year in years:
    print(year)
    url_full = url.format(year)
    response = requests.get(url_full)
    page_content = BeautifulSoup(response.content, "html.parser")
    tbill_table = page_content.find("table", attrs={"class":"t-chart"})
    data = get_table_simple(tbill_table)
    all_query_df = pd.DataFrame(data[2:], columns=data[1]) if all_query_df is None else all_query_df.append(pd.DataFrame(data[2:], columns=data[1]))

2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [5]:
multi_columns = []
for tenure in data[0]:
    for label in np.unique(data[1][1:]):
        multi_columns.append((tenure, label))
all_query_df.set_index("DATE",inplace=True)
all_query_df.columns = multi_columns
all_query_df.index = [dt.datetime.strptime(idx, "%m/%d/%y").date() for idx in all_query_df.index]

In [6]:
for col in all_query_df.columns:
    all_query_df[col] = pd.to_numeric(all_query_df[col], errors='coerce')/100.0

In [7]:
from_, to_ = all_query_df.index.min(), all_query_df.index.max()
reindexed_df = all_query_df.reindex(pd.DatetimeIndex(start=from_, end=to_, freq='1D'))#, method='ffill')
reindexed_df['was_empty'] = reindexed_df.isna().all(1)
reindexed_df = reindexed_df.fillna(method='ffill')


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  


In [8]:
reindexed_df['valid_observation'] = ~reindexed_df['was_empty']

In [9]:
import os
os.chdir("/Users/joaopedroaugusto/Documents/Workspace/Projects/portfolioTools")
from series_handler import DynamicDataFrame, IO_HANDLER, IOHandler
reindexed_df.dynamic.meta = {"name":"T-Bill", "source": "US Treasury"}
reindexed_df.dynamic.writeParquet("RISK_FREE", "USA")

In [17]:
reindexed_df

,"('4 WEEKS', 'BANK DISCOUNT')","('4 WEEKS', 'COUPON EQUIVALENT')","('8 WEEKS', 'BANK DISCOUNT')","('8 WEEKS', 'COUPON EQUIVALENT')","('13 WEEKS', 'BANK DISCOUNT')","('13 WEEKS', 'COUPON EQUIVALENT')","('26 WEEKS', 'BANK DISCOUNT')","('26 WEEKS', 'COUPON EQUIVALENT')","('52 WEEKS', 'BANK DISCOUNT')","('52 WEEKS', 'COUPON EQUIVALENT')",'was_empty','valid_observation'
2002-01-02,0.0171,0.0174,NaN,NaN,0.0171,0.0174,0.0181,0.0185,NaN,NaN,False,True
2002-01-03,0.0170,0.0173,NaN,NaN,0.0170,0.0173,0.0178,0.0182,NaN,NaN,False,True
2002-01-04,0.0169,0.0172,NaN,NaN,0.0169,0.0172,0.0178,0.0182,NaN,NaN,False,True
2002-01-05,0.0169,0.0172,NaN,NaN,0.0169,0.0172,0.0178,0.0182,NaN,NaN,True,False
2002-01-06,0.0169,0.0172,NaN,NaN,0.0169,0.0172,0.0178,0.0182,NaN,NaN,True,False
2002-01-07,0.0168,0.0171,NaN,NaN,0.0166,0.0169,0.0173,0.0177,NaN,NaN,False,True
2002-01-08,0.0167,0.0169,NaN,NaN,0.0165,0.0168,0.0174,0.0178,NaN,NaN,False,True
2002-01-09,0.0165,0.0168,NaN,NaN,0.0165,0.0168,0.0173,0.0177,NaN,NaN,False,True
2002-01-10,0.0164,0.0166,NaN,NaN,0.0165,0.0168,0.0171,0.0175,NaN,NaN,False,True
2002-01-11,0.0160,0.0162,NaN,NaN,0.0155,0.0158,0.0159,0.0162,NaN,NaN,False,True
